# Big G Express: Predicting Derates
In this project, you will be working with J1939 fault code data and vehicle onboard diagnostic data to try and predict an upcoming full derate. 

J1939 is a communications protocol used in heavy-duty vehicles (like trucks, buses, and construction equipment) to allow different electronic control units (ECUs), like the engine, transmission, and brake systems, to talk to each other. Fault codes in this system follow a standard format so that mechanics and diagnostic tools can understand what's wrong, no matter the make or model.

These fault codes have two parts. First, an SPN (Suspect Parameter Number), which identifies what system or component is having the issue. Second, an FMI (Failure Mode Identifier), which explains how the system is failing (too high, too low, short circuit, etc.).

A derate refers to the truck's computer intentionally reducing engine power or speed to protect itself or force the driver to get it serviced. This is a built-in safety measure. A full derate, the main target in this project, means the vehicle is severely limited, requiring a tow for repairs. Full derates are indicated by an SPN of 5246. 

You have been provided with a two files containing the data you will use to make these predictions (J1939Faults.csv and VehicleDiagnosticOnboardData.csv) as well as two files describing some of the contents (DataInfo.docx and Service Fault Codes_1_0_0_167.xlsx) 

Note that in its raw form the data does not have "labels", so you must define what labels you are going to use and create those labels in your dataset. Also, you will likely need to perform some significant feature engineering in order to build an accurate predictor.

There are service locations at (36.0666667, -86.4347222), (35.5883333, -86.4438888), and (36.1950, -83.174722), so you should remove any records in the vicinity of these locations, as fault codes may be tripped when working on the vehicles.

When evaluating the performance of your model, assume that the cost associated with a missed full derate is approximately $4000 in towing and repairs, and the cost of a false positive prediction is about $500 due to having the truck off the road and serviced unnecessarily. While high accuracy or F1 is nice, we are most interested here in saving the company money, so the final metric to evaulate your model should be the cost savings.

**Project Timeline:**

Thursday, May 8: Present preliminary findings to instructors.
Tuesday, May 13: Present final findings to class.

Your presentation should use slides, not code in a notebook. Your final presentation should include at least the following points:
* What features did you use to predict? Report some of the more impactful features using some kind of feature importance metric.
* If you had used the data prior to 2019 to train your model and had been using it from January 1, 2019 onwards, how many full derates would you have caught? How many false positives? What is the net savings or cost of using your model for that time span? Report your estimate here, even if the model would have been a net negative.

In [26]:
#pip install geopy

In [28]:
import pandas as pd
from geopy.distance import geodesic
import numpy as np

**Read in data**

In [4]:
faults = pd.read_csv("../data/J1939Faults.csv")
diagnostics = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv")

C:\Users\jdavis\AppData\Local\Temp\ipykernel_15644\1953169697.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  faults = pd.read_csv("../data/J1939Faults.csv")


**Faults : Cleaning Data**

In [5]:
drop_list = ['ESS_Id', 
             'actionDescription', 
             'ecuSoftwareVersion', 
             'ecuSerialNumber', 
             'ecuModel', 
             'ecuMake', 
             'ecuSource', 
             'faultValue',
             'LocationTimeStamp',
             'MCTNumber']

faults = faults.drop(columns=drop_list)

In [ ]:
service_stations = [
    (36.0666667, -86.4347222),
    (35.5883333, -86.4438888),
    (36.1950, -83.174722)
]

threshold_distance = 1.0  


def is_near_service_station(lat, lon):
    point = (lat, lon)
    for station in service_stations:
        distance = geodesic(point, station).kilometers
        if distance <= threshold_distance:
            return True
    return False


faults['IsServiceStation'] = faults.apply(lambda row: is_near_service_station(row['Latitude'], row['Longitude']), axis=1)

In [58]:
faults['IsServiceStation'].value_counts(normalize = True)

**Diagnostics (Features) : Cleaning Data**

In [10]:
#diagnostics.head()

In [12]:
diagnostics['Value'] = diagnostics['Value'].replace({'FALSE': False, 'TRUE': True})

In [14]:
diagnostics_w = diagnostics.pivot(index='FaultId', columns='Name', values='Value')
features = diagnostics_w.reset_index()
features.columns.name = None

In [65]:
#features.head()

In [67]:
#faults.head()

**Merged Faults and Features**

In [42]:
combined = pd.merge(faults, 
                    features, 
                    left_on='RecordID', 
                    right_on='FaultId',
                    how = 'left')
#combined.head()

,RecordID,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Latitude,Longitude,...,FuelTemperature,IgnStatus,IntakeManifoldTemperature,LampStatus,ParkingBrake,ServiceDistance,Speed,SwitchedBatteryVoltage,Throttle,TurboBoostPressure
0,1,2015-02-21 10:47:13.000,Low (Severity Low) Engine Coolant Level,111,17,True,2,1439,38.857638,-84.626851,...,NaN,False,78.8,1023,True,NaN,0,3276.75,NaN,0
1,2,2015-02-21 11:34:34.000,NaN,629,12,True,127,1439,38.857638,-84.626851,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
2,3,2015-02-21 11:35:31.000,Incorrect Data Steering Wheel Angle,1807,2,False,127,1369,41.421250,-87.767361,...,NaN,NaN,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
3,4,2015-02-21 11:35:33.000,Incorrect Data Steering Wheel Angle,1807,2,True,127,1369,41.421018,-87.767361,...,NaN,True,NaN,1279,NaN,NaN,NaN,NaN,NaN,NaN
4,5,2015-02-21 11:39:41.000,NaN,4364,17,False,2,1674,38.416481,-89.442638,...,NaN,NaN,NaN,16639,NaN,NaN,NaN,NaN,NaN,NaN


In [48]:
combined_filtered = combined[combined['IsServiceStation'] == False]
#combined_filtered['IsServiceStation'].value_counts(normalize = True)

IsServiceStation
False    1.0
Name: proportion, dtype: float64

In [85]:
combined['IsDerateFull'] = combined['spn'] == 5246
#combined[combined['IsDerateFull'] == True]

In [89]:
combined['IsDerateFull'].value_counts(normalize = True)

IsDerateFull
False    0.998994
True     0.001006
Name: proportion, dtype: float64

**Investigate NaN values in both outcomes of response variable:**

*Is there a difference in available data between the two outcomes: IsDerateFull T/F*



In [ ]:
BarometricPressure,
EngineRpm
EngineLoad
IntakeManifoldTemperature
SwitchedBatteryVoltage

In [156]:
feature_columns = [
    'AcceleratorPedal', 'BarometricPressure', 'CruiseControlActive', 'CruiseControlSetSpeed', 
    'DistanceLtd', 'EngineCoolantTemperature', 'EngineLoad', 'EngineOilPressure', 'EngineOilTemperature',
    'EngineRpm', 'EngineTimeLtd', 'FuelLevel', 'FuelLtd', 'FuelRate', 'FuelTemperature', 'IgnStatus', 
    'IntakeManifoldTemperature', 'LampStatus', 'ParkingBrake', 'ServiceDistance', 'Speed', 
    'SwitchedBatteryVoltage', 'Throttle', 'TurboBoostPressure'
]

derate_true = derate_true[feature_columns]
derate_false = derate_false[feature_columns]

nan_count_true = derate_true.isna().sum()
non_nan_count_true = derate_true.count()

nan_count_false = derate_false.isna().sum()
non_nan_count_false = derate_false.count()

nan_percent_true = (nan_count_true / len(derate_true)) * 100
nan_percent_false = (nan_count_false / len(derate_false)) * 100

nan_summary_true = pd.DataFrame({
    'Feature': nan_count_true.index, 
    'NaN Count': nan_count_true.values,
    'Non-NaN Count': non_nan_count_true.values,
    'NaN Percentage (%)_IsDerateFull = True': nan_percent_true.values
})

nan_summary_false = pd.DataFrame({
    'Feature': nan_count_false.index, 
    'NaN Count': nan_count_false.values,
    'Non-NaN Count': non_nan_count_false.values,
    'NaN Percentage (%)_IsDerateFull = False': nan_percent_false.values
})

In [158]:
true_pct = nan_summary_true[['Feature','NaN Percentage (%)_IsDerateFull = True']]
false_pct = nan_summary_false[['Feature','NaN Percentage (%)_IsDerateFull = False']]

nan_comparison = pd.merge(true_pct, false_pct, on = 'Feature')

In [162]:
nan_comparison

,Feature,NaN Percentage (%)_IsDerateFull = True,NaN Percentage (%)_IsDerateFull = False
0,AcceleratorPedal,62.928870,55.195340
1,BarometricPressure,53.138075,50.645286
2,CruiseControlActive,67.447699,51.563306
3,CruiseControlSetSpeed,66.610879,51.434148
4,DistanceLtd,55.230126,50.656415
5,EngineCoolantTemperature,53.138075,50.637277
6,EngineLoad,53.723849,50.674625
7,EngineOilPressure,53.305439,50.622523
8,EngineOilTemperature,54.393305,50.818032
9,EngineRpm,53.054393,50.565701
